In [ ]:
# Cell 1: Imports and Setup
import json

import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import (
    LSTM,
    BatchNormalization,
    Bidirectional,
    Dense,
    Dropout,
    Input,
    Layer,
)
from tensorflow.keras.models import Model

print("✅ Imports loaded successfully!")
print(f"📊 TensorFlow version: {tf.__version__}")
print(f"🎮 GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")
if len(tf.config.list_physical_devices('GPU')) > 0:
    print(f"   GPU: {tf.config.list_physical_devices('GPU')[0].name}")

In [ ]:
# Cell 2: Configuration
TARGET_FRAMES = 64
N_FACE = 468
N_POSE = 33
N_LEFT_HAND = 21
N_RIGHT_HAND = 21
TOTAL_LANDMARKS = N_FACE + N_POSE + N_LEFT_HAND + N_RIGHT_HAND  # 543

# With velocity features: 543 landmarks × 6 coords (x,y,z + vx,vy,vz)
FEATURES_PER_FRAME = TOTAL_LANDMARKS * 6  # 3258
INPUT_SHAPE = (TARGET_FRAMES, FEATURES_PER_FRAME)  # (64, 3258)

# Dataset Paths
ISLR_TRAIN_PATH = "/kaggle/input/wlasl2000-landmarks/train_top200.tfrecord"
ISLR_VAL_PATH = "/kaggle/input/wlasl2000-landmarks/val_top200.tfrecord"
ISLR_MAPPING_PATH = "/kaggle/input/islr-mappings/sign_to_prediction_index_map.json"

# Training settings
USE_AUGMENTATION = True
AUGMENTATION_PROBABILITY = 0.5
USE_VELOCITY_FEATURES = True
USE_BIDIRECTIONAL = True
USE_ATTENTION_POOLING = True

print("✅ Configuration:")
print(f"   Target frames: {TARGET_FRAMES} ({TARGET_FRAMES/30:.1f}s @ 30fps)")
print(f"   Total landmarks: {TOTAL_LANDMARKS}")
print(f"   Input shape: {INPUT_SHAPE}")
print(f"   Augmentation: {'✅ ENABLED' if USE_AUGMENTATION else '❌ DISABLED'}")
print(f"   Velocity features: {'✅ ENABLED' if USE_VELOCITY_FEATURES else '❌ DISABLED'}")
print(f"   BiLSTM: {'✅ ENABLED' if USE_BIDIRECTIONAL else '❌ DISABLED'}")
print(f"   Attention pooling: {'✅ ENABLED' if USE_ATTENTION_POOLING else '❌ DISABLED'}")

In [ ]:
# Cell 3: Load ISLR Mapping
with open(ISLR_MAPPING_PATH, 'r') as f:
    islr_full_mapping = json.load(f)

print(f"✅ ISLR mapping loaded: {len(islr_full_mapping)} words")
print(f"   First 10 words: {list(islr_full_mapping.keys())[:10]}")
print(f"   Label range: {min(islr_full_mapping.values())} - {max(islr_full_mapping.values())}")

# Create reverse mapping
islr_label_to_word = {v: k for k, v in islr_full_mapping.items()}

print("\n🎯 Will train on ~200 words present in TFRecord files")

In [ ]:
# Cell 4: TFRecord Parsing Functions
def parse_tfrecord_example(example_proto):
    feature_description = {
        'video': tf.io.FixedLenFeature([], tf.string),    
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    
    # Decode video landmarks: variable frames x 543 landmarks x 3 coords
    landmarks = tf.io.decode_raw(parsed['video'], tf.float32)     
    landmarks = tf.reshape(landmarks, [-1, TOTAL_LANDMARKS, 3])
    
    label = parsed['label']
    
    return landmarks, label

def resample_to_target_frames(landmarks, target_frames=TARGET_FRAMES):
    """Resample variable-length sequence to fixed 64 frames"""
    current_frames = tf.shape(landmarks)[0]
    
    # If already correct length
    if current_frames == target_frames:
        return landmarks
    
    # If longer: sample evenly spaced frames
    if current_frames >= target_frames:
        indices = tf.linspace(0.0, tf.cast(current_frames-1, tf.float32), target_frames)
        indices = tf.cast(indices, tf.int32)
        landmarks = tf.gather(landmarks, indices)
    else:
        # If shorter: pad with last frame
        padding = target_frames - current_frames
        last_frame = landmarks[-1:]
        padding_frames = tf.tile(last_frame, [padding, 1, 1])
        landmarks = tf.concat([landmarks, padding_frames], axis=0)
    
    return landmarks

print("✅ TFRecord parsing functions ready")

In [ ]:
# Cell 5: Augmentation Functions
def augment_landmarks(landmarks, augment_prob=0.5):
    """
    Apply robust augmentation to landmark sequences
    landmarks shape: (frames, 543, 3)
    Returns: augmented landmarks with same shape
    """
    if np.random.random() > augment_prob:
        return landmarks  # No augmentation
    
    augmented = landmarks.copy()
    
    # 1. Horizontal flip (mirror) - 50% chance
    if np.random.random() < 0.5:
        augmented[:, :, 0] = 1.0 - augmented[:, :, 0]  # Flip x-coordinates
        # Swap left/right hand landmarks
        # Left hand: 468:489, Right hand: 522:543
        left_hand = augmented[:, 468:489, :].copy()
        right_hand = augmented[:, 522:543, :].copy()
        augmented[:, 468:489, :] = right_hand
        augmented[:, 522:543, :] = left_hand
    
    # 2. Spatial scaling (zoom) - 50% chance
    if np.random.random() < 0.5:
        scale = np.random.uniform(0.9, 1.1)  # ±10% scale
        center = np.array([0.5, 0.5, 0.0])  # Center point
        augmented = (augmented - center) * scale + center
    
    # 3. Spatial translation (shift) - 50% chance
    if np.random.random() < 0.5:
        shift_x = np.random.uniform(-0.05, 0.05)  # ±5% shift
        shift_y = np.random.uniform(-0.05, 0.05)
        augmented[:, :, 0] += shift_x
        augmented[:, :, 1] += shift_y
    
    # 4. Temporal speed variation (time warping) - 30% chance
    if np.random.random() < 0.3:
        speed_factor = np.random.uniform(0.85, 1.15)  # ±15% speed
        n_frames = augmented.shape[0]
        new_length = int(n_frames * speed_factor)
        new_length = max(10, min(new_length, n_frames * 2))  # Clamp
        
        # Resample to new length
        old_indices = np.linspace(0, n_frames - 1, n_frames)
        new_indices = np.linspace(0, n_frames - 1, new_length)
        
        resampled = np.zeros((new_length, TOTAL_LANDMARKS, 3), dtype=np.float32)
        for lm in range(TOTAL_LANDMARKS):
            for coord in range(3):
                resampled[:, lm, coord] = np.interp(new_indices, old_indices, augmented[:, lm, coord])
        
        augmented = resampled
    
    # 5. Gaussian noise (small jitter) - 30% chance
    if np.random.random() < 0.3:
        noise = np.random.normal(0, 0.01, augmented.shape).astype(np.float32)
        augmented += noise
    
    # 6. Random frame dropout (simulate occlusion) - 20% chance
    if np.random.random() < 0.2:
        n_frames = augmented.shape[0]
        n_drop = int(n_frames * 0.05)  # Drop 5% of frames
        if n_drop > 0:
            drop_indices = np.random.choice(n_frames, n_drop, replace=False)
            # Interpolate dropped frames from neighbors
            for idx in drop_indices:
                if idx > 0 and idx < n_frames - 1:
                    augmented[idx] = (augmented[idx-1] + augmented[idx+1]) / 2
    
    # Clip to valid range [0, 1]
    augmented = np.clip(augmented, 0.0, 1.0)
    
    return augmented

print("✅ Augmentation functions ready:")
print("   - Horizontal flip (50%)")
print("   - Spatial scaling ±10% (50%)")
print("   - Translation ±5% (50%)")
print("   - Temporal speed ±15% (30%)")
print("   - Gaussian noise (30%)")
print("   - Frame dropout 5% (20%)")

In [ ]:
# Cell 6: Feature Engineering Functions
def add_velocity_features(landmarks):
    """
    Add velocity (frame difference) features
    landmarks shape: (frames, 543, 3)
    Returns: (frames, 543, 6) - original + velocity
    """
    # Calculate velocity (difference between consecutive frames)
    velocity = np.zeros_like(landmarks)
    velocity[1:] = landmarks[1:] - landmarks[:-1]
    velocity[0] = velocity[1]  # First frame uses second frame's velocity
    
    # Concatenate: original (x,y,z) + velocity (vx,vy,vz)
    features = np.concatenate([landmarks, velocity], axis=-1)
    
    return features  # Shape: (frames, 543, 6)

def process_landmarks_to_features(landmarks, apply_augmentation=False):
    """
    Complete processing pipeline:
    1. Augmentation (optional)
    2. Resample to target frames
    3. Add velocity features
    4. Flatten
    """
    # Step 1: Augmentation (if enabled)
    if apply_augmentation and USE_AUGMENTATION:
        landmarks = augment_landmarks(landmarks, AUGMENTATION_PROBABILITY)
    
    # Step 2: Resample to target frames
    landmarks = resample_to_target_frames(tf.constant(landmarks)).numpy()
    
    # Step 3: Add velocity features (if enabled)
    if USE_VELOCITY_FEATURES:
        features = add_velocity_features(landmarks)  # (64, 543, 6)
    else:
        features = landmarks  # (64, 543, 3)
    
    # Step 4: Flatten to (64, 3258) or (64, 1629)
    features_flat = features.reshape(TARGET_FRAMES, -1)
    
    return features_flat

print("✅ Feature engineering functions ready")
if USE_VELOCITY_FEATURES:
    print("   - Velocity features: ENABLED (doubles feature dimension)")
else:
    print("   - Velocity features: DISABLED")

In [ ]:
# Cell 7: Streaming Data Pipeline (No RAM Loading)
def create_tf_dataset(tfrecord_path, batch_size=32, apply_augmentation=False, shuffle=True):
    """Create TensorFlow dataset that streams from disk"""
    
    def parse_and_process(example_proto):
        # Parse TFRecord
        feature_description = {
            'video': tf.io.FixedLenFeature([], tf.string),
            'label': tf.io.FixedLenFeature([], tf.int64),
        }
        
        parsed = tf.io.parse_single_example(example_proto, feature_description)
        
        # Decode landmarks
        landmarks = tf.io.decode_raw(parsed['video'], tf.float32)
        landmarks = tf.reshape(landmarks, [-1, TOTAL_LANDMARKS, 3])
        
        # Resample to target frames
        landmarks = resample_to_target_frames(landmarks)
        
        # Add velocity features
        velocity = tf.concat([
            tf.zeros_like(landmarks[:1]),  # First frame velocity = 0
            landmarks[1:] - landmarks[:-1]  # Frame differences
        ], axis=0)
        
        # Concatenate original + velocity
        features = tf.concat([landmarks, velocity], axis=-1)  # (64, 543, 6)
        
        # Flatten
        features = tf.reshape(features, [TARGET_FRAMES, -1])  # (64, 3258)
        
        # Apply augmentation (if enabled and training)
        if apply_augmentation:
            # Simple TensorFlow augmentation
            if tf.random.uniform(()) < 0.5:  # 50% chance horizontal flip
                # Flip x coordinates (assuming first half of features are x coords)
                features_reshaped = tf.reshape(features, [TARGET_FRAMES, TOTAL_LANDMARKS, 6])
                x_coords = features_reshaped[:, :, 0]  # x coordinates
                x_coords = 1.0 - x_coords  # flip
                features_reshaped = tf.concat([
                    tf.expand_dims(x_coords, -1),
                    features_reshaped[:, :, 1:]
                ], axis=-1)
                features = tf.reshape(features_reshaped, [TARGET_FRAMES, -1])
            
            if tf.random.uniform(()) < 0.3:  # 30% chance add noise
                noise = tf.random.normal(tf.shape(features), stddev=0.01)
                features = features + noise
        
        return features, parsed['label']
    
    # Create dataset
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)
    
    dataset = dataset.map(
        parse_and_process,
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset

# Create streaming datasets (NO RAM LOADING!)
print("🚀 Creating streaming datasets (no RAM loading)...")

train_dataset = create_tf_dataset(
    ISLR_TRAIN_PATH,
    batch_size=32,
    apply_augmentation=True,
    shuffle=True
)

val_dataset = create_tf_dataset(
    ISLR_VAL_PATH,
    batch_size=32,
    apply_augmentation=False,
    shuffle=False
)

print("✅ Streaming datasets created!")
print("   📊 Data streams directly from disk during training")
print("   💾 RAM usage: <2GB (no data preloading)")

# Count samples and get info (sample a few batches)
print("\n🔍 Analyzing dataset structure...")

train_sample_count = 0
val_sample_count = 0
all_labels = set()

# Sample train dataset
for batch_features, batch_labels in train_dataset.take(10):
    train_sample_count += batch_features.shape[0]
    all_labels.update(batch_labels.numpy().tolist())
    if train_sample_count >= 320:  # Stop after ~10 batches
        break

# Sample val dataset  
for batch_features, batch_labels in val_dataset.take(5):
    val_sample_count += batch_features.shape[0]
    all_labels.update(batch_labels.numpy().tolist())
    if val_sample_count >= 160:  # Stop after ~5 batches
        break

# Get feature shape
for batch_features, batch_labels in train_dataset.take(1):
    feature_shape = batch_features.shape[1:]
    break

num_classes = len(all_labels)

print("\n📊 Dataset Info (from samples):")
print(f"   Feature shape per sample: {feature_shape}")
print(f"   Unique labels found: {num_classes}")
print("   Batch size: 32")
print("   Ready for streaming training!")

print("\n🎯 Datasets ready for training!")

In [ ]:
# # Cell 8: Remap Labels to Continuous Range
# # ISLR labels might not be continuous (0-199), so we remap them
# unique_labels_sorted = sorted(np.unique(np.concatenate([y_train, y_val])))

# # Create mapping: old_label -> new_label (0 to num_classes-1)
# old_to_new_label = {old: new for new, old in enumerate(unique_labels_sorted)}
# new_to_old_label = {new: old for old, new in old_to_new_label.items()}

# # Remap labels
# y_train_remapped = np.array([old_to_new_label[label] for label in y_train], dtype=np.int32)
# y_val_remapped = np.array([old_to_new_label[label] for label in y_val], dtype=np.int32)

# # Create final word mappings
# final_label_to_word = {}
# final_word_to_label = {}

# for new_label, old_label in new_to_old_label.items():
#     word = islr_label_to_word.get(old_label, f"unknown_{old_label}")
#     final_label_to_word[str(new_label)] = word
#     final_word_to_label[word] = new_label

# print(f"✅ Labels remapped to continuous range [0-{num_classes-1}]")
# print(f"   Train labels: {y_train_remapped.min()} - {y_train_remapped.max()}")
# print(f"   Val labels: {y_val_remapped.min()} - {y_val_remapped.max()}")
# print(f"\n📚 Vocabulary (first 20 words):")
# for i in range(min(20, num_classes)):
#     print(f"   {i}: {final_label_to_word[str(i)]}")

In [ ]:
# # Cell 9: Data Quality Check
# print("🔍 Data quality check:")
# print(f"\n📊 Training data:")
# print(f"   Shape: {X_train.shape}")
# print(f"   Range: [{X_train.min():.3f}, {X_train.max():.3f}]")
# print(f"   Mean: {X_train.mean():.3f}")
# print(f"   Std: {X_train.std():.3f}")
# print(f"   NaN values: {np.isnan(X_train).sum()}")
# print(f"   Inf values: {np.isinf(X_train).sum()}")

# print(f"\n📊 Validation data:")
# print(f"   Shape: {X_val.shape}")
# print(f"   Range: [{X_val.min():.3f}, {X_val.max():.3f}]")
# print(f"   Mean: {X_val.mean():.3f}")
# print(f"   Std: {X_val.std():.3f}")
# print(f"   NaN values: {np.isnan(X_val).sum()}")
# print(f"   Inf values: {np.isinf(X_val).sum()}")

# print(f"\n✅ Data is clean and ready for training!")

In [ ]:
# Cell 10: Custom Attention Pooling Layer
class AttentionPooling(Layer):
    """Attention-based temporal pooling layer"""
    
    def __init__(self, **kwargs):
        super(AttentionPooling, self).__init__(**kwargs)
    
    def build(self, input_shape):
        # input_shape: (batch, time, features)
        self.attention_weights = Dense(1, activation='tanh', name='attention_weights')
        super(AttentionPooling, self).build(input_shape)
    
    def call(self, inputs):
        # inputs: (batch, time, features)
        # Compute attention scores
        attention_scores = self.attention_weights(inputs)  # (batch, time, 1)
        attention_scores = tf.nn.softmax(attention_scores, axis=1)  # Normalize over time
        
        # Apply attention weights
        weighted = inputs * attention_scores  # (batch, time, features)
        
        # Sum over time dimension
        output = tf.reduce_sum(weighted, axis=1)  # (batch, features)
        
        return output
    
    def get_config(self):
        config = super(AttentionPooling, self).get_config()
        return config

print("✅ Attention pooling layer defined")

In [ ]:
# Cell 11: Build BiLSTM + Attention model (fixed to 200 classes)

# Fixed number of classes (we verified labels 0–199)
NUM_CLASSES = 200

def build_model(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES):
    inputs = Input(shape=input_shape, name="input")  # (64, 3258)
    x = inputs

    # BiLSTM 1 → (None, 64, 512)
    x = Bidirectional(
        LSTM(256, return_sequences=True),
        name="bilstm_1"
    )(x)
    x = BatchNormalization(name="bn_1")(x)
    x = Dropout(0.3, name="dropout_1")(x)

    # BiLSTM 2 → (None, 64, 1024)
    x = Bidirectional(
        LSTM(512, return_sequences=True),
        name="bilstm_2"
    )(x)
    x = BatchNormalization(name="bn_2")(x)
    x = Dropout(0.3, name="dropout_2")(x)

    # BiLSTM 3 → (None, 64, 512)
    x = Bidirectional(
        LSTM(256, return_sequences=True),
        name="bilstm_3"
    )(x)
    x = BatchNormalization(name="bn_3")(x)
    x = Dropout(0.3, name="dropout_3")(x)

    # Attention pooling over time → (None, 512)
    x = AttentionPooling(name="attention_pooling")(x)

    # Dense head
    x = Dense(512, activation="relu", name="dense_1")(x)
    x = BatchNormalization(name="bn_4")(x)
    x = Dropout(0.4, name="dropout_4")(x)

    x = Dense(256, activation="relu", name="dense_2")(x)
    x = Dropout(0.3, name="dropout_5")(x)

    # Output layer: 200 classes
    outputs = Dense(num_classes, activation="softmax", name="output")(x)

    model = Model(inputs, outputs, name="ISLR_BiLSTM_Attention")
    return model

# Build and compile model
model = build_model()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

print("\n✅ Enhanced BiLSTM Model built:")
print(f"   Input: {INPUT_SHAPE}")
print(f"   Output: {NUM_CLASSES} classes")
print(f"   Parameters: {model.count_params():,}")
print(f"   Model size: ~{model.count_params() * 4 / 1024 / 1024:.1f} MB")
print("   Architecture: BiLSTM + Attention")

model.summary()

In [ ]:
# Cell 12: Train Model with Fixed Hyperparameters
print("🚀 Starting training with FIXED hyperparameters...")

# FIXED: Lower learning rate for better convergence
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.0001,  # ← FIXED: 10x lower (was 0.001)
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-7
    ),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Optimized callbacks
checkpoint = ModelCheckpoint(
    '/kaggle/working/lstm_islr200_ultimate_best.weights.h5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True,
    mode='max',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,  # ← FIXED: Lower minimum
    verbose=1,
    cooldown=2
)

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=20,  # ← FIXED: More patient (was 15)
    restore_best_weights=True,
    verbose=1,
    mode='max',
    min_delta=0.001
)

# Performance monitoring callback
class PerformanceCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        import time
        self.epoch_start_time = time.time()
    
    def on_epoch_end(self, epoch, logs=None):
        epoch_time = time.time() - self.epoch_start_time
        lr = float(self.model.optimizer.learning_rate)
        print(f"   ⏱️  Epoch {epoch+1}: {epoch_time:.1f}s | LR: {lr:.2e} | "
              f"Loss: {logs['loss']:.4f} | Acc: {logs['accuracy']:.4f} | "
              f"Val_Loss: {logs['val_loss']:.4f} | Val_Acc: {logs['val_accuracy']:.4f}")

performance_cb = PerformanceCallback()

# Training info
print("\n📊 FIXED Training Configuration:")
print(f"   Model: BiLSTM + Attention ({model.count_params():,} params)")
print("   Optimizer: Adam (lr=0.0001) ← FIXED: 10x lower")
print("   Batch size: 32")
print("   Max epochs: 150")
print("   Early stopping: 20 epochs patience")
print("   LR reduction: factor=0.5, patience=5")
print("   Data: Streaming (56K train, 19K val)")
print("   Expected time: 4-5 hours")
print("   Target accuracy: 80-85%")

print("\n⏳ Starting FIXED training...\n")

# Train with fixed settings
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=150,
    callbacks=[checkpoint, reduce_lr, early_stop, performance_cb],
    verbose=1
)

print("\n🎉 Training complete!")
print(f"   Total epochs: {len(history.history['loss'])}")
print(f"   Best val accuracy: {max(history.history['val_accuracy'])*100:.2f}%")
print(f"   Final train accuracy: {history.history['accuracy'][-1]*100:.2f}%")
print(f"   Final learning rate: {float(model.optimizer.learning_rate):.2e}")

# Save training history
import json

history_dict = {
    'loss': [float(x) for x in history.history['loss']],
    'accuracy': [float(x) for x in history.history['accuracy']],
    'val_loss': [float(x) for x in history.history['val_loss']],
    'val_accuracy': [float(x) for x in history.history['val_accuracy']]
}

with open('/kaggle/working/training_history.json', 'w') as f:
    json.dump(history_dict, f, indent=2)

print("   💾 Training history saved to training_history.json")

In [ ]:
# Cell 12.5: Verify Data Alignment (Run this BEFORE training if you want)
print("🔍 Verifying data alignment...")

# Sample a batch and check
for batch_features, batch_labels in train_dataset.take(1):
    print("\n📊 Data Check:")
    print(f"   Feature shape: {batch_features.shape}")
    print(f"   Label shape: {batch_labels.shape}")
    print(f"   Label range: {batch_labels.numpy().min()} to {batch_labels.numpy().max()}")
    print(f"   Sample labels: {batch_labels.numpy()[:10]}")
    
    # Check model prediction
    predictions = model.predict(batch_features[:5], verbose=0)
    pred_classes = np.argmax(predictions, axis=1)
    
    print("\n🎯 Model Output Check:")
    print(f"   Prediction shape: {predictions.shape}")
    print(f"   Predicted classes: {pred_classes}")
    print(f"   True labels: {batch_labels.numpy()[:5]}")
    print(f"   Prediction confidence: {[f'{np.max(p)*100:.1f}%' for p in predictions[:5]]}")
    
    break

print("\n✅ Data verification complete!")

In [ ]:
# Cell 13: Evaluate Model Performance (Streaming)
print("🔍 Evaluating model performance...")

# Load best weights
model.load_weights('/kaggle/working/lstm_islr200_ultimate_best.weights.h5')

# Evaluate on validation dataset
print("📊 Evaluating on validation set...")
val_results = model.evaluate(val_dataset, verbose=1)
val_loss, val_acc = val_results

print("\n🎯 FINAL RESULTS:")
print(f"   📊 Validation Loss: {val_loss:.4f}")
print(f"   🎯 Validation Accuracy: {val_acc*100:.2f}%")

# Get sample predictions for analysis
print("\n🔍 Sample predictions:")
prediction_count = 0
correct_predictions = 0

for batch_features, batch_labels in val_dataset.take(5):
    predictions = model.predict(batch_features, verbose=0)
    pred_classes = np.argmax(predictions, axis=1)
    
    for i in range(min(10, len(batch_labels))):
        true_label = int(batch_labels[i])
        pred_label = int(pred_classes[i])
        confidence = predictions[i][pred_classes[i]] * 100
        
        status = "✅" if true_label == pred_label else "❌"
        print(f"   {status} True: {true_label:3d} | Pred: {pred_label:3d} ({confidence:.1f}%)")
        
        if true_label == pred_label:
            correct_predictions += 1
        prediction_count += 1
        
        if prediction_count >= 20:
            break
    
    if prediction_count >= 20:
        break

sample_accuracy = correct_predictions / prediction_count * 100
print(f"\n📈 Sample accuracy: {sample_accuracy:.1f}% ({correct_predictions}/{prediction_count})")

print("\n🎉 Model evaluation complete!")

In [ ]:
# Cell 14: Save Model and Deployment Package
print("💾 Saving model and creating deployment package...\n")

# Save model
best_model.save('/kaggle/working/lstm_islr200_ultimate_final.h5')
best_model.save('/kaggle/working/lstm_islr200_ultimate_savedmodel')

# Create deployment configuration
deployment_config = {
    'model_info': {
        'name': 'ISLR 200-Word Ultimate BiLSTM Model',
        'version': '2.0',
        'framework': 'TensorFlow/Keras',
        'architecture': f"{'BiLSTM' if USE_BIDIRECTIONAL else 'LSTM'} + {'Attention' if USE_ATTENTION_POOLING else 'Standard'}",
        'input_shape': list(INPUT_SHAPE),
        'target_frames': TARGET_FRAMES,
        'total_landmarks': TOTAL_LANDMARKS,
        'features_per_frame': FEATURES_PER_FRAME,
        'num_classes': num_classes,
        'training_samples': len(X_train),
        'validation_samples': len(X_val),
        'use_velocity_features': USE_VELOCITY_FEATURES,
        'use_augmentation': USE_AUGMENTATION,
        'use_bidirectional': USE_BIDIRECTIONAL,
        'use_attention': USE_ATTENTION_POOLING
    },
    'performance': {
        'val_accuracy': float(val_acc),
        'val_loss': float(val_loss),
        'top3_accuracy': float(top3_acc),
        'top5_accuracy': float(top5_acc),
        'top10_accuracy': float(top10_acc),
        'expected_realworld_accuracy': f"{val_acc*0.8*100:.1f}-{val_acc*0.9*100:.1f}%"
    },
    'labels': {
        'word_to_label': final_word_to_label,
        'label_to_word': final_label_to_word,
        'num_classes': num_classes
    },
    'preprocessing': {
        'target_frames': TARGET_FRAMES,
        'landmarks_per_frame': TOTAL_LANDMARKS,
        'coordinates_per_landmark': 6 if USE_VELOCITY_FEATURES else 3,
        'input_flattened': True,
        'landmark_order': 'face(468) + left_hand(21) + pose(33) + right_hand(21)',
        'velocity_features': USE_VELOCITY_FEATURES,
        'feature_order': '(x,y,z,vx,vy,vz)' if USE_VELOCITY_FEATURES else '(x,y,z)'
    },
    'improvements': {
        'augmentation': 'Flip, Scale, Translate, Speed, Noise, Dropout',
        'velocity_features': 'Frame-to-frame differences',
        'bidirectional_lstm': USE_BIDIRECTIONAL,
        'attention_pooling': USE_ATTENTION_POOLING,
        'expected_gain': '+20-30% over baseline'
    }
}

# Save deployment config
with open('/kaggle/working/deployment_config.json', 'w') as f:
    json.dump(deployment_config, f, indent=2)

# Save training history
history_data = {
    'loss': [float(x) for x in history.history['loss']],
    'accuracy': [float(x) for x in history.history['accuracy']],
    'val_loss': [float(x) for x in history.history['val_loss']],
    'val_accuracy': [float(x) for x in history.history['val_accuracy']]
}

with open('/kaggle/working/training_history.json', 'w') as f:
    json.dump(history_data, f, indent=2)

# Create README
readme = f"""# ISLR 200-Word Ultimate Sign Language Recognition Model

## 🎯 Model Performance
- **Validation Accuracy:** {val_acc*100:.2f}%
- **Top-5 Accuracy:** {top5_acc*100:.2f}%
- **Training Samples:** {len(X_train):,}
- **Vocabulary:** {num_classes} words

## 🚀 Improvements Over Baseline
- ✅ **Augmentation:** Flip, Scale, Translate, Speed, Noise, Dropout
- ✅ **Velocity Features:** Frame-to-frame motion capture
- ✅ **BiLSTM:** Bidirectional temporal modeling
- ✅ **Attention Pooling:** Learned temporal importance
- 📈 **Expected Gain:** +20-30% over baseline LSTM

## 📊 Model Specifications
- **Input:** {TARGET_FRAMES} frames × {FEATURES_PER_FRAME} features = {INPUT_SHAPE}
- **Architecture:** {'BiLSTM' if USE_BIDIRECTIONAL else 'LSTM'} + {'Attention' if USE_ATTENTION_POOLING else 'Standard'}
- **Parameters:** {model.count_params():,}
- **Model Size:** ~{model.count_params() * 4 / 1024 / 1024:.1f} MB

## 🔧 Usage
```python
import tensorflow as tf
import json

# Load model
model = tf.keras.models.load_model('lstm_islr200_ultimate_final.h5')

# Load config
with open('deployment_config.json') as f:
    config = json.load(f)

# Preprocess landmarks (64 frames, 543 landmarks, 3 coords)
# Add velocity features if enabled
# Flatten to (64, 3258) or (64, 1629)

# Predict
prediction = model.predict(landmarks)
predicted_label = np.argmax(prediction)
predicted_word = config['labels']['label_to_word'][str(predicted_label)]
```

## 📦 Files
- `lstm_islr200_ultimate_final.h5` - Keras model
- `lstm_islr200_ultimate_savedmodel/` - TensorFlow SavedModel
- `deployment_config.json` - Complete configuration
- `training_history.json` - Training metrics
- `README.md` - This file

## 🎓 Training Details
- **Dataset:** ISLR 200 words
- **Augmentation:** {AUGMENTATION_PROBABILITY*100:.0f}% probability
- **Optimizer:** Adam (lr=0.001)
- **Batch Size:** 64
- **Epochs:** {len(history.history['loss'])}
- **Early Stopping:** Patience 20

## 📝 Notes
- Velocity features {'ENABLED' if USE_VELOCITY_FEATURES else 'DISABLED'}
- Input requires {TARGET_FRAMES} frames (2.13s @ 30fps)
- Expected real-world accuracy: {val_acc*0.8*100:.1f}-{val_acc*0.9*100:.1f}%
"""

with open('/kaggle/working/README.md', 'w') as f:
    f.write(readme)

print("✅ DEPLOYMENT PACKAGE CREATED:")
print("   📦 lstm_islr200_ultimate_final.h5")
print("   📦 lstm_islr200_ultimate_savedmodel/")
print("   📦 deployment_config.json")
print("   📦 training_history.json")
print("   📦 README.md")

print("\n🎯 MODEL SUMMARY:")
print("   🎪 Dataset: ISLR 200 words")
print(f"   📊 Training: {len(X_train):,} samples")
print(f"   🎯 Accuracy: {val_acc*100:.1f}%")
print(f"   🥇 Top-5: {top5_acc*100:.1f}%")
print(f"   📚 Vocabulary: {num_classes} words")
print(f"   ⚡ Input: {TARGET_FRAMES} frames ({TARGET_FRAMES/30:.1f}s)")
print(f"   💾 Size: ~{model.count_params() * 4 / 1024 / 1024:.1f} MB")

print("\n🚀 IMPROVEMENTS APPLIED:")
print("   ✅ Augmentation (6 types)")
print("   ✅ Velocity features (+motion)")
print("   ✅ BiLSTM (bidirectional)")
print("   ✅ Attention pooling")
print("   📈 Expected gain: +20-30%")

print("\n📥 DOWNLOAD ALL FILES FROM /kaggle/working/")
print("\n🎉 TRAINING COMPLETE! Ultimate model ready for deployment.")